# Preprocesamiento - Segmentación de Oraciones con NLTK

**Objetivo:** Dividir los documentos en oraciones y luego aplicar el preprocesamiento (tokenización, POS, lematización) a cada oración. Usaremos `nltk.send_tokenize` y adaptaremos nuestro pipeline para procesar oracion por oracion.


In [60]:
import sys
import os
import time
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups

In [61]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(f"Añadido '{module_path}' a sys.path")

In [62]:
from src.functions import batch_generator, preprocess_text_step1, get_wordnet_pos

In [63]:
global lematizer
lemmatizer = WordNetLemmatizer()

In [64]:
newsgroups_data = fetch_20newsgroups(subset='all',
                                     remove=('headers', 'footers', 'quotes'),
                                     data_home='./data/20newsgroups_cache')
documents = newsgroups_data.data

In [65]:
def lemmatize_sentence_nltk(sentence_text):
    """Tokeniza, etiqueta POS y lematiza una única oración (string)."""
    tokens = nltk.word_tokenize(sentence_text, language='english')
    pos_tags = nltk.pos_tag(tokens)
    lemmas = []
    for word, tag in pos_tags:
        if word.isalnum():
            wordnet_tag = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, pos=wordnet_tag)
            lemmas.append(lemma)
    return lemmas

In [66]:
# función principal que orquesta el pipeline para 1 documento
def segment_and_process_document(doc_text):
    """
    Normaliza, segmenta en oraciones y lematiza cada oración de un documento.

    Args:
        doc_text (str): Texto crudo del documento.

    Returns:
        list[list[str]]: Lista de oraciones, donde cada oración es una lista de lemas.
    """
    # normalizamos los documentos previamente a la segmentación
    normalized_doc = preprocess_text_step1(doc_text)

    # usamos el segmentor de oraciones de nltk para dividir el texto en oraciones
    sentences = nltk.sent_tokenize(normalized_doc, language='english')

    # lematizamos cada oración y la agregamos a una lista
    processed_doc = []
    for sentence in sentences:
        lemmatized_sentence = lemmatize_sentence_nltk(sentence)
        # nos aseguramos de no agregar  listas vacías si una oración solo contenía puntuación
        if lemmatized_sentence:
            processed_doc.append(lemmatized_sentence)

    return processed_doc

***recordar**: Los tokens se pueden ver como una palabra en una oracion o una oracion en un parrafo*

`nltk.sent_tokenize` lo que hace es tokenizar el texto en oraciones, a diferencia de `nltk.word_tokenize` que tokeniza por palabras en una oracion.

Por que hacemos esto? Porque muchos modelos de lenguajes procesan la informacion oracion por oracion.

In [67]:
batch_size = 10
print(f"\nObteniendo el primer lote (batch_size = {batch_size})...")
batch_gen = batch_generator(documents, batch_size)
first_batch = next(batch_gen, [])


Obteniendo el primer lote (batch_size = 10)...


In [68]:
start_proc_time = time.time()
# processed_batch ahora contendrá una lista de listas de listas
# [doc1, doc2, ...] donde doc1 = [sent1_lemmas, sent2_lemmas, ...] y sent1_lemmas = [lemma1, lemma2, ...]
processed_batch_segmented = [segment_and_process_document(doc) for doc in first_batch]
end_proc_time = time.time()
print(f"Lote procesado en {end_proc_time - start_proc_time:.4f} segundos.")

Lote procesado en 0.0261 segundos.


# Mostramos los resultados

In [69]:
num_examples_to_show = 6
print(f"\nMostrando los primeros {num_examples_to_show} ejemplos procesados:")

for i in range(min(num_examples_to_show, len(first_batch))):
    print(f"\n--- Documento {i+1} ---")
    original_text = first_batch[i]
    processed_doc = processed_batch_segmented[i]

    print(f"  Original ({len(original_text)} chars):\n'{original_text}'")
    print(f"\n  Número de oraciones detectadas: {len(processed_doc)}")
    print("\n  Primeras oraciones procesadas (lemas):")
    for j, sentence_lemmas in enumerate(processed_doc):
        print(f"    Oración {j+1} ({len(sentence_lemmas)} lemas): {sentence_lemmas}")



Mostrando los primeros 6 ejemplos procesados:

--- Documento 1 ---
  Original (712 chars):
'

I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!

'

  Número de oraciones detectadas: 9

  Primeras oraciones procesadas (lemas):
    Oración 1 (27 lemas): ['i', 'be', 'sure', 'some', 'bashers', 'of', 'pen', 'fan', 'be', 'pretty', 'confuse', 'a